<a href="https://colab.research.google.com/github/sundar-nallalagappan/LLM-Qwen-Finetuning/blob/main/Qwen3_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
print("sairam")

sairam


In [18]:
pip install -U transformers accelerate bitsandbytes torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 579.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 141.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,          # critical
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

model.eval()


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm

In [3]:
SYSTEM_PROMPT = """You are a senior 5G radio network optimization engineer.
You analyze drive-test KPIs, radio parameters, interference, mobility, and scheduling.
Select the single most likely root cause.
Return ONLY the final answer in the format \\boxed{C<number>}.
Do not include explanations.
"""


In [5]:
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.pad_token_id = tokenizer.eos_token_id
print(tokenizer.pad_token, tokenizer.eos_token)
print(tokenizer.pad_token_id,tokenizer.eos_token_id)

<|endoftext|> <|im_end|>
151643 151645


In [6]:
messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": "Hello"}
    ]
messages

[{'role': 'system',
  'content': 'You are a senior 5G radio network optimization engineer.\nYou analyze drive-test KPIs, radio parameters, interference, mobility, and scheduling.\nSelect the single most likely root cause.\nReturn ONLY the final answer in the format \\boxed{C<number>}.\nDo not include explanations.\n'},
 {'role': 'user', 'content': 'Hello'}]

In [31]:
t = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        padding=True
    ).to(model.device)
t.eq(tokenizer.pad_token_id)

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False]], device='cuda:0')

In [7]:
def predict_root_cause(question: str):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    attention_mask = inputs.ne(tokenizer.pad_token_id)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            attention_mask=attention_mask,
            max_new_tokens=20,
            do_sample=False,      # deterministic
            temperature=0.0,
            top_p=1.0
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded


In [8]:
import re

def extract_answer(text):
    match = re.search(r"\\boxed\{(C\d)\}", text)
    if match:
        return match.group(1)
    else:
        return "C1"  # safe fallback


In [34]:
sample_question = """

Analyze the 5G wireless network drive-test user plane data and engineering parameters.\nIdentify the reason for the throughput dropping below 600Mbps in certain road sections.\nFrom the following 8 potential root causes, select the most likely one and enclose its number in \\boxed{{}} in the final answer.\n\nC1: The serving cell's downtilt angle is too large, causing weak coverage at the far end.\nC2: The serving cell's coverage distance exceeds 1km, resulting in over-shooting.\nC3: A neighboring cell provides higher throughput.\nC4: Non-colocated co-frequency neighboring cells cause severe overlapping coverage.\nC5: Frequent handovers degrade performance.\nC6: Neighbor cell and serving cell have the same PCI mod 30, leading to interference.\nC7: Test vehicle speed exceeds 40km\/h, impacting user throughput.\nC8: Average scheduled RBs are below 160, affecting throughput.\n\nGiven:\n- The default electronic downtilt value is 255, representing a downtilt angle of 6 degrees. Other values represent the actual downtilt angle in degrees.\n\nBeam Scenario and Vertical Beamwidth Relationships:\n- When the cell's Beam Scenario is set to Default or SCENARIO_1 to SCENARIO_5, the vertical beamwidth is 6 degrees.\n- When the cell's Beam Scenario is set to SCENARIO_6 to SCENARIO_11, the vertical beamwidth is 12 degrees.\n- When the cell's Beam Scenario is set to SCENARIO_12 or above, the vertical beamwidth is 25 degrees.\n\nUser plane drive test data as follows\uff1a\n\nTimestamp|Longitude|Latitude|GPS Speed (km\/h)|5G KPI PCell RF Serving PCI|5G KPI PCell RF Serving SS-RSRP [dBm]|5G KPI PCell RF Serving SS-SINR [dB]|5G KPI PCell Layer2 MAC DL Throughput [Mbps]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 1 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 2 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 3 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 4 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 5 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 1 Filtered Tx BRSRP [dBm]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 2 Filtered Tx BRSRP [dBm]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 3 Filtered Tx BRSRP [dBm]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 4 Filtered Tx BRSRP [dBm]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 5 Filtered Tx BRSRP [dBm]|5G KPI PCell Layer1 DL RB Num (Including 0)\n2025-05-07 15:23:52.000000|128.188169|32.579273|1|712|-77.0|15.93|1351.25|258|71|284|-|-|-90.49|-95.82|-102.2|-|-|210.91\n2025-05-07 15:23:53.000000|128.18814|32.579223|2|71|-80.97|6.6|366.57|258|712|129|-|-|-88.72|-93.26|-105.47|-|-|211.23\n2025-05-07 15:23:54.000000|128.188117|32.579174|16|71|-85.5|1.81|334.0|258|712|-|-|-|-84.41|-96.46|-|-|-|210.99\n2025-05-07 15:23:55.000000|128.188103|32.579113|14|71|-88.21|5.4|431.94|712|258|-|-|-|-88.04|-93.72|-|-|-|195.26\n2025-05-07 15:23:56.000000|128.188088|32.579075|19|71|-78.45|13.59|566.34|712|258|284|-|-|-87.97|-98.7|-105.92|-|-|209.0\n2025-05-07 15:23:57.000000|128.188073|32.579033|22|258|-77.45|11.45|1308.74|712|71|284|129|-|-89.56|-93.31|-104.18|-110.78|-|204.95\n2025-05-07 15:23:58.000000|128.188051|32.579002|16|258|-81.66|10.06|1310.14|712|71|284|129|-|-84.85|-93.3|-102.87|-114.09|-|210.13\n2025-05-07 15:23:59.000000|128.188073|32.578995|26|258|-79.13|16.32|1023.36|712|71|284|129|-|-86.75|-93.71|-104.01|-107.26|-|208.06\n2025-05-07 15:24:00.000000|128.188036|32.578984|33|258|-74.09|19.8|770.83|712|71|284|129|-|-89.29|-96.4|-100.21|-114.11|-|207.07\n2025-05-07 15:24:01.000000|128.188036|32.578984|25|258|-71.09|19.62|1014.75|712|71|-|-|-|-90.14|-92.93|-|-|-|205.57\n\n\nEngeneering parameters data as follows\uff1a\n\ngNodeB ID|Cell ID|Longitude|Latitude|Mechanical Azimuth|Mechanical Downtilt|Digital Tilt|Digital Azimuth|Beam Scenario|Height|PCI|TxRx Mode|Max Transmit Power|Antenna Model\n0034038|4|128.188273|32.579129|50|6|4|0|DEFAULT|5.0|712|64T64R|34.9|NR AAU 2\n0033164|27|128.217634|32.58072|240|3|10|0|DEFAULT|29.7|71|32T32R|34.9|NR AAU 1\n0034038|2|128.188282|32.579099|140|10|4|0|DEFAULT|5.0|258|64T64R|34.9|NR AAU 2\n0034038|24|128.18769|32.57791|40|44|9|10|SCENARIO_9|56.0|284|64T64R|34.9|NR AAU 2\n0033164|14|128.189481|32.581043|150|20|255|0|DEFAULT|28.7|129|64T64R|34.9|NR AAU 2
"""

raw_output = predict_root_cause(sample_question)
print("RAW OUTPUT:\n", raw_output)

answer = extract_answer(raw_output)
print("FINAL ANSWER:", answer)


<>:3: SyntaxWarning: invalid escape sequence '\/'
<>:3: SyntaxWarning: invalid escape sequence '\/'
/tmp/ipython-input-829750774.py:3: SyntaxWarning: invalid escape sequence '\/'
  Analyze the 5G wireless network drive-test user plane data and engineering parameters.\nIdentify the reason for the throughput dropping below 600Mbps in certain road sections.\nFrom the following 8 potential root causes, select the most likely one and enclose its number in \\boxed{{}} in the final answer.\n\nC1: The serving cell's downtilt angle is too large, causing weak coverage at the far end.\nC2: The serving cell's coverage distance exceeds 1km, resulting in over-shooting.\nC3: A neighboring cell provides higher throughput.\nC4: Non-colocated co-frequency neighboring cells cause severe overlapping coverage.\nC5: Frequent handovers degrade performance.\nC6: Neighbor cell and serving cell have the same PCI mod 30, leading to interference.\nC7: Test vehicle speed exceeds 40km\/h, impacting user throughput.

RAW OUTPUT:
 system
You are a senior 5G radio network optimization engineer.
You analyze drive-test KPIs, radio parameters, interference, mobility, and scheduling.
Select the single most likely root cause.
Return ONLY the final answer in the format \boxed{C<number>}.
Do not include explanations.

user


Analyze the 5G wireless network drive-test user plane data and engineering parameters.
Identify the reason for the throughput dropping below 600Mbps in certain road sections.
From the following 8 potential root causes, select the most likely one and enclose its number in \boxed{{}} in the final answer.

C1: The serving cell's downtilt angle is too large, causing weak coverage at the far end.
C2: The serving cell's coverage distance exceeds 1km, resulting in over-shooting.
C3: A neighboring cell provides higher throughput.
C4: Non-colocated co-frequency neighboring cells cause severe overlapping coverage.
C5: Frequent handovers degrade performance.
C6: Neighbor cell and serving cell have

In [9]:
import pandas as pd
data = pd.read_csv("phase_1_test.csv")
data.shape

(864, 2)

In [10]:
data.sample(5)

,ID,question
158,ID_NONLL91ZSF,Analyze the 5G wireless network drive-test use...
426,ID_PKIW6TN8KS,Analyze the 5G wireless network drive-test use...
653,ID_PJJOCQFX2F,Analyze the 5G wireless network drive-test use...
536,ID_SBRFBRC6HX,Analyze the 5G wireless network drive-test use...
495,ID_A03W45JFTT,Analyze the 5G wireless network drive-test use...


In [11]:
data[['Qwen3-32B', 'Qwen2.5-7B-Instruct', 'Qwen2.5-1.5B-Instruct']] = 'placeholder'
data.iloc[0]

,0
ID,ID_XLWWVM40IW
question,Analyze the 5G wireless network drive-test use...
Qwen3-32B,placeholder
Qwen2.5-7B-Instruct,placeholder
Qwen2.5-1.5B-Instruct,placeholder


In [13]:
answer = []
for idx, question in enumerate(data['question']):
  #print(question)
  print(f"Processing --> {idx}")
  raw_output = predict_root_cause(question)
  #print("RAW OUTPUT:\n", raw_output)

  response = extract_answer(raw_output)
  #print("FINAL ANSWER:", response)
  answer.append(response)


Processing --> 0
Processing --> 1
Processing --> 2
Processing --> 3
Processing --> 4
Processing --> 5
Processing --> 6
Processing --> 7
Processing --> 8
Processing --> 9
Processing --> 10
Processing --> 11
Processing --> 12
Processing --> 13
Processing --> 14
Processing --> 15
Processing --> 16
Processing --> 17
Processing --> 18
Processing --> 19
Processing --> 20
Processing --> 21
Processing --> 22
Processing --> 23
Processing --> 24
Processing --> 25
Processing --> 26
Processing --> 27
Processing --> 28
Processing --> 29
Processing --> 30
Processing --> 31
Processing --> 32
Processing --> 33
Processing --> 34
Processing --> 35
Processing --> 36
Processing --> 37
Processing --> 38
Processing --> 39
Processing --> 40
Processing --> 41
Processing --> 42
Processing --> 43
Processing --> 44
Processing --> 45
Processing --> 46
Processing --> 47
Processing --> 48
Processing --> 49
Processing --> 50
Processing --> 51
Processing --> 52
Processing --> 53
Processing --> 54
Processing --> 55
Pr

In [15]:
answer[1:10]

['C8', 'C8', 'C8', 'C8', 'C8', 'C8', 'C8', 'C8', 'C8']

In [17]:
data['Qwen2.5-7B-Instruct'] = answer

In [19]:
data['Qwen2.5-7B-Instruct'].value_counts()

,count
Qwen2.5-7B-Instruct,
C8,848
C6,15
C4,1


In [22]:
data1 = data.drop('question', axis=1)

In [23]:
data1.to_csv("baseline_inference.csv", index=False)

In [29]:
rows=[]
for _, row in data1.iterrows():
  base_id = row["ID"]
  Qwen2_5_7B_Instruct = row['Qwen2.5-7B-Instruct']
  Qwen3_32B = row['Qwen3-32B']
  Qwen2_5_1_5B_Instruct = row['Qwen2.5-1.5B-Instruct']

  for k in range(1,5):
    rows.append(
        {"ID":f'{base_id}_{k}',
         "Qwen3-32B":Qwen3_32B,
        "Qwen2.5-7B-Instruct" : Qwen2_5_7B_Instruct,
         "Qwen2.5-1.5B-Instruct":Qwen2_5_1_5B_Instruct
        })
data2 = pd.DataFrame(rows)

In [30]:
data2.to_csv("baseline_inference1.csv", index=False)

In [1]:
#Merge
import pandas as pd
result = pd.read_csv("baseline_inference1.csv")
result

,ID,Qwen3-32B,Qwen2.5-7B-Instruct,Qwen2.5-1.5B-Instruct
0,ID_XLWWVM40IW_1,placeholder,C8,placeholder
1,ID_XLWWVM40IW_2,placeholder,C8,placeholder
2,ID_XLWWVM40IW_3,placeholder,C8,placeholder
3,ID_XLWWVM40IW_4,placeholder,C8,placeholder
4,ID_3WB8KX32W3_1,placeholder,C8,placeholder
...,...,...,...,...
3451,ID_C0UQ0QAUS6_4,placeholder,C8,placeholder
3452,ID_OVT01Y45B4_1,placeholder,C8,placeholder
3453,ID_OVT01Y45B4_2,placeholder,C8,placeholder
3454,ID_OVT01Y45B4_3,placeholder,C8,placeholder


In [2]:
sub = pd.read_csv("SampleSubmission.csv")
sub

,ID,Qwen3-32B,Qwen2.5-7B-Instruct,Qwen2.5-1.5B-Instruct
0,ID_XLWWVM40IW_1,placeholder,"Based on the provided data, the most likely ro...",placeholder
1,ID_XLWWVM40IW_2,placeholder,"Based on the provided data, the most likely ro...",placeholder
2,ID_XLWWVM40IW_3,placeholder,"Based on the provided data, the most likely ro...",placeholder
3,ID_XLWWVM40IW_4,placeholder,"Based on the provided data, the most likely ro...",placeholder
4,ID_3WB8KX32W3_1,placeholder,"Based on the provided data, the most likely ro...",placeholder
...,...,...,...,...
6903,ID_EWGKK32YWU_4,placeholder,placeholder,placeholder
6904,ID_MSMYVLJJIZ_1,placeholder,placeholder,placeholder
6905,ID_MSMYVLJJIZ_2,placeholder,placeholder,placeholder
6906,ID_MSMYVLJJIZ_3,placeholder,placeholder,placeholder


In [36]:
merged = pd.merge(sub, result, on='ID', how='left')
merged1 = merged.drop(['Qwen3-32B_x', 'Qwen2.5-7B-Instruct_x', 'Qwen2.5-1.5B-Instruct_x'], axis=1)
merged1.shape

(6908, 4)

In [37]:
mask = merged1['Qwen2.5-7B-Instruct_y'].notna()
merged1.loc[mask, 'Qwen2.5-7B-Instruct_y'] = (
    merged1.loc[mask, 'Qwen2.5-7B-Instruct_y']
    .apply(lambda c: f"Based on the provided data, the most likely root cause is:\\boxed{{{c}}}")
)

In [38]:
#merged1['Qwen2.5-7B-Instruct'] = merged1['Qwen2.5-7B-Instruct_y'].apply(lambda c: f"Based on the provided data, the most likely root cause is:\n\\boxed{{{c}}}")

In [39]:
merged1[mask]

,ID,Qwen3-32B_y,Qwen2.5-7B-Instruct_y,Qwen2.5-1.5B-Instruct_y
0,ID_XLWWVM40IW_1,placeholder,"Based on the provided data, the most likely ro...",placeholder
1,ID_XLWWVM40IW_2,placeholder,"Based on the provided data, the most likely ro...",placeholder
2,ID_XLWWVM40IW_3,placeholder,"Based on the provided data, the most likely ro...",placeholder
3,ID_XLWWVM40IW_4,placeholder,"Based on the provided data, the most likely ro...",placeholder
4,ID_3WB8KX32W3_1,placeholder,"Based on the provided data, the most likely ro...",placeholder
...,...,...,...,...
3451,ID_C0UQ0QAUS6_4,placeholder,"Based on the provided data, the most likely ro...",placeholder
3452,ID_OVT01Y45B4_1,placeholder,"Based on the provided data, the most likely ro...",placeholder
3453,ID_OVT01Y45B4_2,placeholder,"Based on the provided data, the most likely ro...",placeholder
3454,ID_OVT01Y45B4_3,placeholder,"Based on the provided data, the most likely ro...",placeholder


In [40]:
merged1.rename(columns={'Qwen3-32B_y':'Qwen3-32B', 'Qwen2.5-1.5B-Instruct_y':'Qwen2.5-1.5B-Instruct', 'Qwen2.5-7B-Instruct_y': 'Qwen2.5-7B-Instruct'}, inplace=True)
#merged1.drop('Qwen2.5-7B-Instruct_y', axis=1, inplace=True)
merged1.shape

(6908, 4)

In [41]:
merged1.sample(2)

,ID,Qwen3-32B,Qwen2.5-7B-Instruct,Qwen2.5-1.5B-Instruct
4679,ID_3ZY67E37JW_4,NaN,NaN,NaN
6283,ID_XP5PQB7VII_4,NaN,NaN,NaN


In [42]:
merged1 = merged1.fillna('placeholder')
merged2 = merged1[['ID', 'Qwen3-32B', 'Qwen2.5-7B-Instruct', 'Qwen2.5-1.5B-Instruct']]
merged2

,ID,Qwen3-32B,Qwen2.5-7B-Instruct,Qwen2.5-1.5B-Instruct
0,ID_XLWWVM40IW_1,placeholder,"Based on the provided data, the most likely ro...",placeholder
1,ID_XLWWVM40IW_2,placeholder,"Based on the provided data, the most likely ro...",placeholder
2,ID_XLWWVM40IW_3,placeholder,"Based on the provided data, the most likely ro...",placeholder
3,ID_XLWWVM40IW_4,placeholder,"Based on the provided data, the most likely ro...",placeholder
4,ID_3WB8KX32W3_1,placeholder,"Based on the provided data, the most likely ro...",placeholder
...,...,...,...,...
6903,ID_EWGKK32YWU_4,placeholder,placeholder,placeholder
6904,ID_MSMYVLJJIZ_1,placeholder,placeholder,placeholder
6905,ID_MSMYVLJJIZ_2,placeholder,placeholder,placeholder
6906,ID_MSMYVLJJIZ_3,placeholder,placeholder,placeholder


In [43]:
merged2.to_csv('merged2.csv', index=False)